In [1]:
import shelve

## Code Shelf

In [2]:
shelf = shelve.open('CodeShelf')

## Add or Update

In [6]:
value = """
T_SAMPLES_PER_CLASS = 3
V_SAMPLES_PER_CLASS = 3

def make_dataset_samples(ds, n_samples_per_class):
    samples = list(map(lambda _: [], range(N_CLASSES)))
    total_samples = 0
    for image,label in ds:
        if len(samples[label]) < n_samples_per_class:
            samples[label].append(image)
            total_samples += 1
        
        if total_samples >= n_samples_per_class*N_CLASSES:
            break
    return samples

def show_samples(samples, samples_per_class, title='Samples'):
    fig, axes = plt.subplots(N_CLASSES, samples_per_class, figsize=(samples_per_class*3, N_CLASSES*2))

    for row in range(N_CLASSES):
        for col in range(samples_per_class):
            axes[row, col].imshow(samples[row][col])
            axes[row, col].axis('off')
            axes[row, col].set_title('C ' + str(row))
    
    plt.tight_layout()
    fig.suptitle(title, fontsize=16)
    
    return fig

T_SAMPLES = make_dataset_samples(train_ds, T_SAMPLES_PER_CLASS)
V_SAMPLES = make_dataset_samples(val_ds, V_SAMPLES_PER_CLASS)
"""

key = 'show_dataset_samples'

print(shelf.get(key))
shelf[key] = value
shelf.sync()

None


## Enumerate

In [7]:
keys = list(shelf.keys())
print(len(keys), keys)

8 ['plot_history', 'model_summary', 'run_summary', 'tf_imports', 'basic_image_augmentations', 'image_randaugment', 'run_summary_multioutput', 'show_dataset_samples']


## Query

In [5]:
print(shelf['run_summary'])


def plot_history(h):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5), facecolor='w', edgecolor='k')

    axes[0].plot(h['loss'], label='Loss')
    axes[0].plot(h['val_loss'], label='Validation Loss')

    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Losses')

    axes[1].plot(h['accuracy'], label='Accuracy')
    axes[1].plot(h['val_accuracy'], label='Validation Accuracy')

    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Accuracies')

    fig.legend()

def run_summary(h):
    plot_history(h)
    print(max(h['accuracy']), '|', max(h['val_accuracy']))

